In [30]:
import pandas as pd
import numpy as np

import re 
import random
from imblearn.over_sampling import RandomOverSampler
from scipy.sparse import hstack

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import warnings 
warnings.filterwarnings("ignore")


In [31]:
df = pd.read_csv('data.csv')

In [32]:
df.dropna(inplace = True)

In [33]:
status_counts = df['status'].value_counts()

In [41]:
random_statements = df.groupby('status')['statement'].apply(lambda x: x.sample(n=1).iloc[0])

In [35]:
df.rename(columns={'statement': 'original_statement'}, inplace=True)

In [36]:
df['statement']=df['original_statement'].str.lower()


In [37]:
def remove_patterns(text):

    '''
    This function removes all URLs, markdown-style links,
    handels, and punctuation/other special characters
    '''
    
    text = re.sub(r'http[s]?://\S+', '', text)

    text = re.sub(r'\[.*?\]\(.*?\)', '', text)

    text = re.sub(r'@\w+', '', text)

    text = re.sub(r'[^\w\s]', '', text)
    
    return text.strip()

df['statement'] = df['statement'].apply(remove_patterns)

In [38]:
df['tokens'] = df['statement'].apply(word_tokenize)

In [ ]:
stemmer = PorterStemmer()

def stem_tokens(tokens):
    return ' '.join(stemmer.stem(str(token)) for token in tokens)

df['tokens_stemmed'] = df['tokens'].apply(stem_tokens)

,Unnamed: 0,original_statement,status,statement,tokens,tokens_stemmed
0,0,oh my gosh,Anxiety,oh my gosh,"[oh, my, gosh]",oh my gosh
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety,trouble sleeping confused mind restless heart ...,"[trouble, sleeping, confused, mind, restless, ...",troubl sleep confus mind restless heart all ou...
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,all wrong back off dear forward doubt stay in ...,"[all, wrong, back, off, dear, forward, doubt, ...",all wrong back off dear forward doubt stay in ...
3,3,I've shifted my focus to something else but I'...,Anxiety,ive shifted my focus to something else but im ...,"[ive, shifted, my, focus, to, something, else,...",ive shift my focu to someth els but im still w...
4,4,"I'm restless and restless, it's been a month n...",Anxiety,im restless and restless its been a month now ...,"[im, restless, and, restless, its, been, a, mo...",im restless and restless it been a month now b...


In [40]:
X = df[['tokens_stemmed', 'num_of_characters', 'num_of_sentences']]
y = df['status']

KeyError: "['num_of_characters', 'num_of_sentences'] not in index"